In [1]:
!pip install openai langchain tiktoken langchain-openai langchain-community -q


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip


## Если используете ключ от OpenAI, запустите эту ячейку 👇


In [15]:
from langchain_openai import ChatOpenAI
import os
from getpass import getpass


# os.environ['OPENAI_API_KEY'] = "Введите ваш OpenAI API ключ"
os.environ['OPENAI_API_KEY'] = getpass(prompt='Введите ваш OpenAI API ключ')

# Инициализируем языковую модель
llm = ChatOpenAI(temperature=0.0)

## Если используете ключ из курса, запустите эти ячейки 👇


In [1]:
!wget https://raw.githubusercontent.com/a-milenkin/LLM_practical_course/main/notebooks/utils.py

--2025-04-30 22:42:01--  https://raw.githubusercontent.com/a-milenkin/LLM_practical_course/main/notebooks/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11316 (11K) [text/plain]
Saving to: ‘utils.py.1’

utils.py.1          100%[===================>]  11.05K  --.-KB/s    in 0.02s   

2025-04-30 22:42:03 (540 KB/s) - ‘utils.py.1’ saved [11316/11316]



In [18]:
from utils import ChatOpenAI
from getpass import getpass

#course_api_key= "Введите ваш API ключ с курса"
course_api_key = getpass(prompt='Введите API ключ')

# Инициализируем языковую модель
llm = ChatOpenAI(temperature=0.0, course_api_key=course_api_key)

## Задание 2.1.8 LLM — сила 💪, ML — могила 🧟

In [19]:
import pandas as pd
from tqdm import tqdm

In [20]:
df = pd.read_csv('https://stepik.org/media/attachments/lesson/1084297/submission100lines.csv')

In [21]:
df.head()

,amount,text_id,text
0,2,14205200,Сниму жилье. 500-600 рублей сутки. Эконом клас...
1,6,319097075,ищем жилье в центре недалеко от моря с 23.07-0...
2,4,98881311,Ищем жилье на период с 18-28июля..на две семьи...
3,4,44587027,2 семьи по 3 человека (2 взрослых и ребенок) с...
4,3,352802829,Сниму недорогое жилье в лазаревском на 6 чел 3...


In [22]:
# Напишите шаблон промпта, указав инструкцию, вопрос и выходной индикатор

prompt_template = """
Инструкция: Прочитай приведённый ниже текст и определи, сколько человек в нём упоминается. Ответь только числом, без дополнительных слов и символов.

Context: {text_input}

Question: Сколько человек упоминается в этом тексте?

Answer: csv файл, содержащий три столбца - amount, text_id, text.

(Столбец amount должен быть числового типа)
"""

In [28]:
amount_list = [] # Список, где будем хранить ответы модели

for text_input in tqdm(df['text']):
    prompt = prompt_template.format(text_input=text_input) # Добавляем сообщение в промпт
    amount = llm.invoke(prompt).content # Ответ модели
    amount_list.append(amount) # Добавляем ответ в список
    # break # Для отладки. Уберите, когда убедитесь, что на одном примере работает

100%|██████████| 100/100 [01:23<00:00,  1.19it/s]


In [29]:
# перед записью ответа в датафрейм, проверьте какого типа ответы
type(amount_list[0])

str

In [30]:
amount_list

['1',
 '5',
 '7',
 '6',
 '6',
 '2',
 '1',
 '4',
 '4',
 '3',
 '4',
 '3',
 '5',
 '5',
 '6',
 '4',
 '2',
 '4',
 '2',
 '6',
 '3',
 '4',
 '2',
 '4',
 '3',
 '2',
 '6',
 '3',
 '4',
 '3',
 '6',
 '2',
 '4',
 '6',
 '8',
 '1',
 '3',
 '2',
 '2',
 '3',
 '4',
 '3',
 '3',
 '3',
 '6',
 '2',
 '3',
 '4',
 '4',
 '3',
 '3',
 '3',
 '4',
 '4',
 '4',
 '2',
 '2',
 '2',
 '2',
 '2',
 '3',
 '4',
 '11',
 '2',
 '3',
 '2',
 '2',
 '5',
 '3',
 '4',
 '3',
 '2',
 '3',
 '2',
 '2',
 '4',
 '5',
 '0',
 '3',
 '1',
 '4',
 '4',
 '4',
 '1',
 '2',
 '7',
 '5',
 '4',
 '3',
 '3',
 '7',
 '5',
 '5',
 '4',
 '12',
 '9',
 '1',
 '0',
 '2',
 '0']

In [25]:
# Чтобы система зачла ответы на Степике, надо перевести их в числа
# Если не знаете как, спросите у ChatGPT)

In [31]:
df['amount'] = amount_list # Создаём новый столбец из ответов модели

In [36]:
import polars as pl

In [38]:
df = pl.from_pandas(df)

In [42]:
df.with_columns(pl.col('amount').cast(pl.Int64)).write_csv('2_1_8_solution.csv')

In [41]:
df

amount,text_id,text
str,i64,str
"""1""",14205200,"""Сниму жилье. 500-600 рублей су…"
"""5""",319097075,"""ищем жилье в центре недалеко о…"
"""7""",98881311,"""Ищем жилье на период с 18-28ию…"
"""6""",44587027,"""2 семьи по 3 человека (2 взрос…"
"""6""",352802829,"""Сниму недорогое жилье в лазаре…"
…,…,…
"""9""",233661411,"""Добрый день! Ищем жилье на 9 ч…"
"""1""",77792435,"""Здравствуйте. Ищем жильё в пер…"
"""0""",102311098,"""Добрый день! Ищем жилье в Лаза…"


In [ ]:
df.to_csv('2_1_8_solution.csv', index=False) # Сохраняем файл, отправляем на Stepik, получаем баллы :)